<a href="https://colab.research.google.com/github/pdhoot16/DataViz/blob/master/Interactive_Cars_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import altair as alt
import numpy as np
import pandas as pd

#from google.colab import files
#uploaded = files.upload()

In [0]:
import io
df2 = pd.read_excel("https://raw.githubusercontent.com/pdhoot16/DataViz/master/04cars%20data.xls")

In [4]:
df2.head()

,VehicleName,TypeOfVehicle,Company,Country,Sedan,Sports_Car,SUV,Wagon,Minivan,Pickup,...,City_MPG,Hwy_MPG,Weight,Wheel_Base,Len,Width,Lat,Long,Dimension,Avg_MPG
0,Acura 3.5 RL 4dr,Sedan,Acura,Japan,1,0,0,0,0,0,...,18,24,3880,115,197,72,36.2048° N,138.2529° E,197x72,21
1,Acura 3.5 RL w/Navigation 4dr,Sedan,Acura,Japan,1,0,0,0,0,0,...,18,24,3893,115,197,72,36.2048° N,138.2529° E,197x72,21
2,Acura MDX,SUV,Acura,Japan,0,0,1,0,0,0,...,17,23,4451,106,189,77,36.2048° N,138.2529° E,189x77,20
3,Acura NSX coupe 2dr manual S,Sports Car,Acura,Japan,0,1,0,0,0,0,...,17,24,3153,100,174,71,36.2048° N,138.2529° E,174x71,20.5
4,Acura RSX Type S 2dr,Sedan,Acura,Japan,1,0,0,0,0,0,...,24,31,2778,101,172,68,36.2048° N,138.2529° E,172x68,27.5


In [9]:
interval = alt.selection_multi(fields=['Company'])
brush = alt.selection(type='interval', resolve='global')
carName = alt.selection_single(fields=['VehicleName'])
carType = alt.selection_single(fields=['TypeOfVehicle'])
brush_tool = alt.selection(type='single', encodings=['x'])
AWDselection = alt.selection_multi(fields=['AWD'])
RWDselection = alt.selection_multi(fields=['RWD'])

color1 = alt.condition(interval,
                      alt.Color('Company:N',legend=None),
                      alt.value('lightgray'))

color2 = alt.condition(brush,
                      alt.Color('Company:N',legend=None),
                      alt.value('lightgray'))

color3 = alt.condition(carName,
                      alt.Color('Company:N',legend=None),
                      alt.value('lightgray'))


color4 = alt.condition(carType,
                    alt.Color('Company:N', legend = None),
                    alt.value('lightgray'))


#####################

points = alt.Chart(df2).mark_point().encode(
  x='Retail_Price',
  y='HP',
  tooltip=['VehicleName','HP','Retail_Price'],
  color=color2
    #alt.condition(interval, 'Company', alt.value('lightgray'))
).add_selection(
    brush
).transform_filter(
    interval
).transform_filter(
    carType
).transform_filter(
    carName
).transform_filter(
    brush_tool
).transform_filter(
    AWDselection
).transform_filter(
    RWDselection
)


#####################

histogram = alt.Chart(df2).mark_bar().encode(
  x='Company',
  y='count()',
  tooltip=['Company','count()'],
  color=color1
).transform_filter(
    brush
).transform_filter(
    carName
).transform_filter(
    carType
).transform_filter(
    brush_tool
).transform_filter(
    AWDselection
).transform_filter(
    RWDselection
).properties(
  selection=interval
)

#####################

bar = alt.Chart(df2).mark_bar().encode(
    x="Country",
    y="Retail_Price",
    color=color3,
    tooltip=['VehicleName','Country','Retail_Price']
).properties(
  width = 800
).transform_filter(
    brush
).transform_filter(
    interval
).transform_filter(
    carType
).transform_filter(
    brush_tool
).transform_filter(
    AWDselection
).transform_filter(
    RWDselection
).add_selection(
    carName
)

#####################

carTypeBar = alt.Chart(df2).mark_bar().encode(
    x=alt.X('count()'),
    y='TypeOfVehicle',
    tooltip=[alt.X('count()'),'Country','TypeOfVehicle'] ,
    color=color4
).transform_filter(
    brush
).transform_filter(
    interval
).transform_filter(
    carName
).transform_filter(
    brush_tool
).add_selection(
  carType
).transform_filter(
    AWDselection
).transform_filter(
    RWDselection
).properties(
    width=700,
    height=300
)


#####################


# Define the base chart, with the common parts of the
# background and highlights
base = alt.Chart().mark_bar().encode(
    x=alt.X(alt.repeat('column'), type='quantitative', bin=alt.Bin(maxbins=20)),
    y='Retail_Price',
    tooltip=['Retail_Price']
).properties(
    width=200,
    height=200
)

# blue background with selection
background = base.properties(selection=brush_tool)

# yellow highlights on the transformed data
highlight = base.encode(
    color=alt.value('goldenrod')
).transform_filter(
    brush_tool
).transform_filter(
    brush
).transform_filter(
    interval
).transform_filter(
    carName
).transform_filter(
    carType
).transform_filter(
    AWDselection
).transform_filter(
    RWDselection
)

# layer the two charts & repeat
common_bar = alt.layer(
    background,
    highlight,
    data=df2
).repeat(column=["Cyl", "Engine_Size", "City_MPG","Hwy_MPG","Wheel_Base","Len","Weight","Width"])


################



awdPoint = alt.Chart(df2
).mark_point(filled=True,size=1500
).encode(
    y=alt.Y('AWD:O', axis=alt.Axis(orient='right', title='AWD')),
      color=alt.condition(AWDselection,
                        'AWD:N',
                        alt.value('lightgray'),
                         legend=None),
    tooltip=['AWD'],
    
).add_selection(
  AWDselection
).transform_filter(
    brush
).transform_filter(
    interval
).transform_filter(
    carType
).transform_filter(
    brush_tool
).transform_filter(
    carName
).transform_filter(
    RWDselection
).properties(
    width=100,
    height=150
)

################

rwdPoint = alt.Chart(df2
).mark_point(filled=True,size=1500
).encode(
    y=alt.Y('RWD:O', axis=alt.Axis(orient='right', title='RWD')),
      color=alt.condition(RWDselection,
                        'RWD:N',
                        alt.value('lightgray'),
                         legend=None),
    tooltip=['RWD'],
    
).add_selection(
  RWDselection
).transform_filter(
    brush
).transform_filter(
    interval
).transform_filter(
    carType
).transform_filter(
    brush_tool
).transform_filter(
    carName
).transform_filter(
    AWDselection
).properties(
    width=100,
    height=150
)


################


dashboard = (points | histogram | (awdPoint & rwdPoint)) & (bar | carTypeBar) & common_bar
dashboard

VConcatChart({
  vconcat: [HConcatChart({
    hconcat: [Chart({
      data:                                    VehicleName TypeOfVehicle     Company  \
      0                             Acura 3.5 RL 4dr         Sedan       Acura   
      1                Acura 3.5 RL w/Navigation 4dr         Sedan       Acura   
      2                                    Acura MDX           SUV       Acura   
      3                 Acura NSX coupe 2dr manual S    Sports Car       Acura   
      4                         Acura RSX Type S 2dr         Sedan       Acura   
      5                                 Acura TL 4dr         Sedan       Acura   
      6                                Acura TSX 4dr         Sedan       Acura   
      7                             Audi A4 1.8T 4dr         Sedan       Audi    
      8                              Audi A4 3.0 4dr         Sedan       Audi    
      9                  Audi A4 3.0 convertible 2dr         Sedan       Audi    
      10                Audi A4 3.0 Quattro 4dr auto         Sedan       Audi    
      11              Audi A4 3.0 Quattro 4dr manual         Sedan       Audi    
      12         Audi A4 3.0 Quattro convertible 2dr         Sedan       Audi    
      13                 Audi A41.8T convertible 2dr         Sedan       Audi    
      14               Audi A6 2.7 Turbo Quattro 4dr         Sedan       Audi    
      15                             Audi A6 3.0 4dr         Sedan       Audi    
      16                   Audi A6 3.0 Avant Quattro         Wagon       Audi    
      17                     Audi A6 3.0 Quattro 4dr         Sedan       Audi    
      18                     Audi A6 4.2 Quattro 4dr         Sedan       Audi    
      19                       Audi A8 L Quattro 4dr         Sedan       Audi    
      20                               Audi RS 6 4dr    Sports Car       Audi    
      21                       Audi S4 Avant Quattro         Wagon       Audi    
      22                         Audi S4 Quattro 4dr         Sedan       Audi    
      23         Audi TT 1.8 convertible 2dr (coupe)    Sports Car       Audi    
      24       Audi TT 1.8 Quattro 2dr (convertible)    Sports Car       Audi    
      25         Audi TT 3.2 coupe 2dr (convertible)    Sports Car       Audi    
      26                               BMW 325Ci 2dr         Sedan         BMW   
      27                   BMW 325Ci convertible 2dr         Sedan         BMW   
      28                                BMW 325i 4dr         Sedan         BMW   
      29                               BMW 325xi 4dr         Sedan         BMW   
      ..                                         ...           ...         ...   
      397                              Toyota Tacoma        Pickup     Toyota    
      398            Toyota Tundra Access Cab V6 SR5        Pickup     Toyota    
      399               Toyota Tundra Regular Cab V6        Pickup     Toyota    
      400                    Volkswagen Golf GLS 4dr         Sedan  Volkswagen   
      401              Volkswagen GTI 1.8T 2dr hatch         Sedan  Volkswagen   
      402                        Volkswagen Jetta GL         Wagon  Volkswagen   
      403               Volkswagen Jetta GLI VR6 4dr         Sedan  Volkswagen   
      404               Volkswagen Jetta GLS TDI 4dr         Sedan  Volkswagen   
      405         Volkswagen New Beetle GLS 1.8T 2dr         Sedan  Volkswagen   
      406  Volkswagen New Beetle GLS convertible 2dr         Sedan  Volkswagen   
      407                 Volkswagen Passat GLS 1.8T         Wagon  Volkswagen   
      408                  Volkswagen Passat GLS 4dr         Sedan  Volkswagen   
      409       Volkswagen Passat GLX V6 4MOTION 4dr         Sedan  Volkswagen   
      410                       Volkswagen Passat W8         Wagon  Volkswagen   
      411           Volkswagen Passat W8 4MOTION 4dr         Sedan  Volkswagen   
      412                     Volkswagen Phaeton 4dr         Sedan  Volkswag

In [0]:
dashboard.save('dashboard.html')

from google.colab import files
files.download('dashboard.html')